**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install datasets

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset
from matplotlib.colors import LogNorm
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch

## Klasifikácia MNIST číslic

Tento príklad bude ilustrovať, ako sa dá zostrojiť jednoduchá konvolučná sieť na klasifikáciu obrazu na dátovej množine MNIST obsahujúcej rukou písané číslice.

### Načítanie dátovej množiny

Začneme načítaním dátovej množiny MNIST. Tento krok bude veľmi jednoduchý, pretože balíček `datasets` od HugginFace obsahuje vstavanú funkciu, ktorá dáta stiahne aj načíta. Zavoláme iba funkciu `load_dataset`, pričom ako dátovú množinu špecifikujeme `"mnist"`. Získame takto dátovú množinu, ktorá je už rozdelená na tréningovú a testovaciu časť – získať ich vieme pomocou `dataset['train']` a `dataset['test']`.

Každá časť obsahuje dva zoznamy `'image'` a `'label'`; `'image'` je zoznam `PIL` obrázkov, ktoré je možné prviesť nanumpy polia pomocou funkcie `np.asarray`. Pod kľúčom `'label'` nájdete označenia tried (požadované výstupy).

Pri načítavaní dát sa musíme uistiť, že tenzory sú správne škálované a majú správny tvar. Naše údaje sa skladajú z obrázkov $28 \times 28$ s jedným farebným kanálom. V `PyTorch`-i sú farebné kanály reprezentované 1. rozmerom tenzora (pričom 0. rozmer je rozmer dávky). Náš tenzor má tvar `(dávka, 28, 28)` a my potrebujeme, aby mal tvar `(dávka, 1, 28, 28)`, takže zavoláme `.unsqueeze(1)`. Napokon si všimneme, že rozsah hodnôt je od 0 do 255 a preškálujeme ich do rozsahu od 0 po 1, t. j. delíme ich číslom 255.



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = load_dataset("mnist")

X_train_np = np.asarray([np.asarray(img) for img in dataset['train']['image']])
Y_train_np = np.asarray(dataset['train']['label'])
X_test_np = np.asarray([np.asarray(img) for img in dataset['test']['image']])
Y_test_np = np.asarray(dataset['test']['label'])

X_train = torch.as_tensor(X_train_np).to(device)
Y_train = torch.as_tensor(Y_train_np).to(device)
X_test = torch.as_tensor(X_test_np).to(device)
Y_test = torch.as_tensor(Y_test_np).to(device)

X_train = X_train.unsqueeze(1) / 255.0
X_test = X_test.unsqueeze(1) / 255.0

Niekoľko náhodne zvolených vzoriek z tréningovej množiny si zobrazme.



In [ ]:
num_rows = 4; num_cols = 4
fig, axes = plt.subplots(num_rows, num_cols)

for row in axes:
    for ax in row:
        ax.imshow(X_train_np[np.random.randint(0,
                    len(X_train_np)-1)],
                  cmap='Greys')
        ax.set_xticks([])
        ax.set_yticks([])

### Dátové množiny a data loader-y

Doteraz sme s dátami pracovali v režime úplných dávok: do siete sme v každom kroku vložili vždy celú dátovú množinu. To je, samozrejme možné iba vtedy, ak je dátová množina dostatočne malá na to, aby sa do pamäte zmestila celá naraz. Ak so sieťou pracujeme na GPU, vieme ju potom spustiť na všetkých dátach paralelne, takže beh je výpočtovo efektívny.

Pri hlbokom učení je však väčšina dátových množín príliš veľká na to, aby sa zmestila do pamäte naraz – môžu mať pokojne desiatky alebo stovky gigabajtov a niektoré sú ešte väčšie. Ak je vaša dátová množina taká veľká, je samozrejme nevyhnutné, aby bolo možné načítavať dáta z pevného disku za pochodu a trénovať v režime mini-dávok. V PyTorch-i sa tento aspekt hlbokého učenia rieši pomocou objektov `Dataset` a `DataLoader`.

#### Trieda Dataset

Trieda `Dataset` poskytuje unifikované rozhranie na prístup k dátam. Existuje množstvo rôznych tried, ktoré dedia z `Dataset`, a implementujú podporu pre rôzne formáty dátových množín, napr. `ImageNet`, `VOCDetection`, `Cityscapes`, `CelebA`. Existuje dokonca aj o niečo všeobecnejšia trieda `ImageFolder`, ktorá jednoducho načítava obrázky a označenia tried z priečinka.

Na definovanie vlastnej dátovej množiny by sme potrebovali implementovať nasledujúce rozhranie:

```
class CustomDataset(Dataset):
    def __init__(self, ...)
        ...

    def __len__(self):
        """
        Returns the number of samples in the dataset.
        """

        ...

    def __getitem__(self, idx):
        """
        Returns the sample at index idx from the dataset.
        """

        ...
```
#### Trieda DataLoader

Data loader dostane ako argument dátovú množinu a je zodpovedný za zostavovanie mini-dávok z nej, zabezpečuje, aby boli dáta korektne zamiešané a pod. Typicky nie je potrebné implementovať vlastný data loader — vo väčšine prípadov stačí použiť triedu `DataLoader` z `torch.utils.data`, napr. takto:

```
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
```
#### Náš príklad a TensorDataset

Dátová množina MNIST je opäť dostatočne malá na to, aby sa zmestila do pamäte celá naraz. S dátovými množinami ako je táto môžeme použiť triedu `TensorDataset`, ktorá iba zabalí existujúci tenzor do `DataSet` rozhrania a umožní ho použiť s DataLoader-mi. 



In [ ]:
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=True)

### Zostavenie konvolučnej siete

Pri zostavovaní konvolučnej siete väčšinou postupujeme tak, že preštudujeme literatúru týkajúcu sa podobných úloh a na základe toho vytvoríme podobnú architektúru neurónovej siete pre náš problém (a prípadne ju doladíme).

Keďže dátová množina MNIST nie je až taká náročná, ukážeme si na nej ešte o čosi jednoduchší prístup:

* Budeme za sebou radiť bloky konvolučných vrstiev, ReLU funkcií a združovacích vrstiev.
* Pokračovať budeme dovtedy, kým sa rozmer vstupného obrazu dostatočne nezníži.
* Následne zaradíme do siete jednu alebo viacero klasických lineárnych vrstiev s ReLU aktivačnými funkciami.
Aby sa nám ľahšie sledovalo, aký rozmer majú dáta po aplikácii jednotlivých vrstiev, neobalíme si všetky vrstvy hneď do tried, ale s nimi budeme najprv voľne experimentovať. Z dátovej množiny si vyberieme zopár vzoriek, ktoré použijeme ako pokusný vstup.



In [ ]:
y = X_train[:5].to('cpu')

Teraz si teda vytvorme prvý blok a aplikujme ho na tenzor `y`. Ako prvú vytvoríme 2D konvolučnú vrstvu pomocou triedy `nn.Conv2d`. Treba pri tom špecifikovať niekoľko parametrov: konkrétne počet vstupných a výstupných kanálov a veľkosť konvolučného jadra. Počet vstupných kanálov bude samozrejme 1, pretože, ako sme už spomenuli, pracujeme s jedným farebným kanálom. Počet výstupných kanálov je jedným z hyperparametrov nášho modelu – začneme s hodnotou 16 keďže naše dáta sú také jednoduché. Za zmienku stojí, že v typickej konvolučnej sieti sa rozmery príznakových máp zvyknú v neskorších vrstvách zmenšovať, ale počet kanálov naopak narastá (intuícia je, že čím hlbšia vrstva, tým abstraktnejšie – a početnejšie – sú koncepty, ktoré reprezentuje).

Konvolučné jadrá môžu mať tiež rôzne veľkosti, ale všeobecné odporúčania vychádzajúce z empirických výsledkov hovoria, že jadrá rozmeru $3 \times 3$ zvyknú fungovať dobre. Snahou je vyhnúť sa tomu, aby jadrá boli príliš veľké, pretože s čím väčšími maticami budeme pracovať, tým dlhšie bude trvať, kým ich medzi sebou vynásobíme.

Po konvolučnej vrstve aplikujeme ReLU aktivačnú funkciu a maximálne združovanie, ktorému je opäť potrebné nastaviť eľkosť jadra. Pri združovaní platí, že čím väčšia je veľkosť jadra, tým rapídnejšie dáta podvzorkovávame. Budeme preto používať malé jadro rozmeru $2 \times 2$. Mnohé moderné architektúry sa v poslednom čase združovacích vrstiev zbavili úplne a namiesto nich používajú na podvzorkovanie väčší krok alebo dilatáciu v konvolučných vrstvách.



In [ ]:
conv1 = nn.Conv2d(
    in_channels=1, out_channels=8,
    kernel_size=(3, 3))

y = conv1(y)
y = torch.relu(y)
y = torch.max_pool2d(y, kernel_size=(2, 2))

Potom, ako sme skonštruovali prvý blok, overme si, aký mal vplyv na rozmer našich dát.



In [ ]:
np.product(y.shape[1:])

Ukazuje sa, že naše dáta sú stále príliš vysokorozmerné a budeme ich rozmer musieť ešte o čosi znížiť. Aplikujme teda ešte jeden blok. Všimnite si, že hoci príznaková mapa sa zmenšuje, počet kanálov zvyšujeme.



In [ ]:
conv2 = nn.Conv2d(8, 16, (3, 3))
y = conv2(y)
y = torch.relu(y)
y = torch.max_pool2d(y, (2, 2))

In [ ]:
np.product(y.shape[1:])

Po aplikácií druhého bloku je počet rozmerov omnoho rozumnejší. Môžeme preto aktuálny výstup zalomiť do vektora (2-rozmernému obraz zmeníme tvar tak, aby sa z neho stal 1-rozmerný vektor) a aplikujme zopár štandardných lineárnych vrstiev s ReLU aktivačnými funkciami. Znovu si pritom dajme pozor, aby sa rozmer dát znižoval postupne a zmena nebola z jednej vrstvy na druhú príliš drastická. Výstupná vrstva bude mať 10 výstupných neurónov, pretože klasifikujeme do 10 tried: na číslice. Ako aktivačnú funckiu použijeme softmax funkciu.



In [ ]:
y = torch.flatten(y, 1)

fc1 = nn.Linear(400, 128)
y = fc1(y)
y = torch.relu(y)

fc2 = nn.Linear(128, 10)
y = fc2(y)

y.shape

Keď sme teda navrhli celú architektúru, potrebujeme ju znovu obaliť do triedy. Ako zvyčajne, vrstvy s parametrami treba vytvoriť už v konštruktore `__init__` a potom použiť vo funkcii `forward`. 



In [ ]:
class Net(nn.Module):
    def __init__(self, num_outputs):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, (3, 3))
        self.conv_acti1 = nn.PReLU()
        
        self.conv2 = nn.Conv2d(8, 16, (3, 3))
        self.conv_acti2 = nn.PReLU()

        self.fc1 = nn.Linear(400, 128)
        self.fc_acti1 = nn.PReLU()

        self.fc2 = nn.Linear(128, num_outputs)
        self.dropout = nn.Dropout(0.3)

    def forward(self, y):
        y = self.conv1(y)
        y = self.conv_acti1(y)
        y = torch.max_pool2d(y, kernel_size=(2, 2))
        y = self.dropout(y)
        
        y = self.conv2(y)
        y = self.conv_acti2(y)
        y = torch.max_pool2d(y, kernel_size=(2, 2))
        y = self.dropout(y)
        
        y = torch.flatten(y, 1)
        
        y = self.fc1(y)
        y = self.fc_acti1(y)
        y = self.dropout(y)

        y = self.fc2(y)
        return y

### Konštrukcia a tréning klasifikátora

Naša tréningová slučka bude teraz trochu iná, keďže používame `Dataset` a `DataLoader` objekty.

Po novom budeme pracovať s dvoma vnorenými slučkami:

* Vonkajšia iteruje cez epochy;
* Vnútorná iteruje cez minidávky v rámci tej istej epochy;
Všimnite si, že teraz zaznamenávame chybu pre každú minidávku. V dôsledku toho bude krivka učenia o niečo viac zašumená – gradienty sú, samozrejme, stabilnejšie, keď sa akumulujú v rámci celej dátovej množiny než keď sa akumulujú len v rámci menšej minidávky.



In [ ]:
num_outputs = 10
model = Net(num_outputs).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_train = []

for epoch in range(50):
    model.train()

    for X_batch, Y_batch in train_dataloader:
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        
        y_batch = model(X_batch)
        loss = criterion(y_batch, Y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_train.append(loss.item())

    if epoch % 5 == 0:
        print(f"epoch {epoch}, loss: {np.mean(loss_train[-20:])}")

print(f"epoch {epoch}, loss: {np.mean(loss_train[-20:])}")

In [ ]:
plt.plot(loss_train)
plt.xlabel("step")
plt.ylabel("loss")
plt.grid(ls='--')

### Testovanie

Napokon aplikujeme opäť našu štandardnú testovaciu procedúru pre klasifikátory: zobrazíme maticu zámen a správnosť na testovacej množine.

#### Na tréningových dátach



In [ ]:
model.eval()
with torch.no_grad():
    y_train_logit = model(X_train)
    y_train = y_train_logit.argmax(dim=1)

cm = pd.crosstab(
    Y_train.cpu().numpy(),
    y_train.cpu().numpy(),
    rownames=['actual'],
    colnames=['predicted']
)
print(cm, "\n")

acc = accuracy_score(Y_train.cpu().numpy(), y_train.cpu().numpy())
print("Accuracy = {}".format(acc))

#### Na testovacích dátach



In [ ]:
model.eval()
with torch.no_grad():
    y_test_logit = model(X_test)
    y_test = y_test_logit.argmax(dim=1)

cm = pd.crosstab(
    Y_test.cpu().numpy(),
    y_test.cpu().numpy(),
    rownames=['actual'],
    colnames=['predicted']
)
print(cm, "\n")

acc = accuracy_score(Y_test.cpu().numpy(), y_test.cpu().numpy())
print("Accuracy = {}".format(acc))